### Extracción de características

In [1]:
import cv2
import numpy as np
import os
import math
import statistics
import csv

from skimage import io, color, img_as_ubyte
from skimage.feature import graycomatrix, graycoprops

light_yellow = np.array([0, 120, 20], np.uint8)
dark_yellow = np.array([175, 255, 255], np.uint8)

path = 'fruit_dataset/test/'
file_list = os.listdir(path)
print(len(file_list))

hue_feature = []
cva_feature = []
entropy_feature = []
energy_feature = []

for i in range(len(file_list)):
    img = cv2.imread(path + file_list[i])
    print('('+str(i)+'): '+path + file_list[i])

    # Conversión a espacio HSV
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)

    # Hue feature
    h_mean = np.array(h).mean()
    positives = h_mean > 0  # Ignoramos los 0 que corresponden al blanco
    if positives:
        h_mean = h_mean[positives].mean()

    print('h_mean:', h_mean)
    hue_feature.append(h_mean)

    # Center of gravity and circle variance
    mask = cv2.inRange(hsv_img, light_yellow, dark_yellow)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    distances = []
    for c in contours:
        area = cv2.contourArea(c)
        if area > 5000:
            M = cv2.moments(c)
            if M["m00"] == 0:
                M["m00"] = 1
            x = int(M["m10"]/M["m00"])
            y = int(M['m01']/M['m00'])
            centroide = np.array((x, y))

            nuevoContorno = cv2.convexHull(c)
            distances.extend([np.linalg.norm(centroide - np.array((c.item(i), c.item(i+1)))) for i in range(0, len(c), 2)])

    if distances:
        mean = statistics.mean(distances)
        st_dev = statistics.pstdev(distances)
        cva = st_dev / mean
    else:
        mean = 0.0
        st_dev = 0.0
        cva = 0.0

    cva_feature.append(cva)

    print('mean:', mean)
    print('stdev:', st_dev)

    # Entropy
    rgbImg = io.imread(path + file_list[i])
    grayImg = img_as_ubyte(color.rgb2gray(rgbImg))

    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    properties = ['energy', 'homogeneity']

    glcm = graycomatrix(grayImg, distances=distances, angles=angles, symmetric=True, normed=True)

    feats = np.hstack([graycoprops(glcm, prop).ravel() for prop in properties])
    entropy = -np.sum(glcm*np.log2(glcm + (glcm==0)))
    print("entropy:", entropy)
    entropy_feature.append(entropy)

    # Energy
    energy = graycoprops(glcm, 'energy')[0]
    print("energy:", energy)
    energy_feature.append(energy)

# Guardar las características en un archivo CSV
output_file = 'caracteristicas_test.csv'
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Hue', 'CVA', 'Entropy', 'Energy'])
    for i in range(len(file_list)):
        writer.writerow([hue_feature[i], cva_feature[i], entropy_feature[i], energy_feature[i]])

cv2.destroyAllWindows()


202
(0): fruit_dataset/test/fruit_100.jpg
h_mean: 71.17953491210938
mean: 65.3587200212113
stdev: 12.82343538084434
entropy: 37.180919804936
energy: [0.20309343 0.18852118 0.19964419 0.18772843]
(1): fruit_dataset/test/fruit_1000.jpg
h_mean: 13.653427124023438
mean: 83.44195818487647
stdev: 1.3612301813653394
entropy: 26.784981869885073
energy: [0.48311272 0.47651081 0.48270537 0.47673434]
(2): fruit_dataset/test/fruit_102.jpg
h_mean: 81.10633850097656
mean: 67.50384606923976
stdev: 23.925486882034644
entropy: 46.51960099399126
energy: [0.0940302  0.0756346  0.07725427 0.07583969]
(3): fruit_dataset/test/fruit_104.jpg
h_mean: 28.649002075195312
mean: 112.20891090159017
stdev: 4.989892178836822
entropy: 40.69335589562402
energy: [0.08452991 0.08097656 0.08605264 0.08121573]
(4): fruit_dataset/test/fruit_113.jpg
h_mean: 10.001358032226562
mean: 91.5210205787702
stdev: 1.8144978947166053
entropy: 25.613071957370163
energy: [0.49957988 0.49381922 0.4993366  0.49385286]
(5): fruit_dataset/t